In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt 
import pandas as pd
import common as cm

# Part 1: Information Gain

Important note: this exercise uses Pandas (for data manipulation and analysis) and Graphviz (for graph-drawing) libraries. 

This exercise consists of 3 parts. Complete the first part to get a mark of 3.0, the first two parts to get 4.0, complete all assignments to get 5.0. 

1.1 ) There are 10 objects (data) characterized with 5 binary attributes:

In [2]:
attributeNames = ["attr 1", "attr 2", "attr 3", "attr 4", "attr 5"]

data = pd.DataFrame(
    [
        [1, 0, 1, 1, 1],
        [1, 1, 0, 0, 1],
        [0, 1, 1, 1, 1],
        [1, 0, 1, 0, 1],
        [1, 0, 0, 1, 1],
        [0, 0, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 0, 1, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 0, 1, 1],
    ],
    columns=attributeNames,
)

1.2) Each object is assigned to either a class "0" or "1". The assignments are as follows (cl):

In [3]:
data["cl"] = [1, 1, 1, 0, 0, 1, 1, 1, 0, 0]

Hint: How one can read data (columns) in Pandas:

In [4]:
print(data["cl"], "\n")
print(list(data["cl"]), "\n")
print(set(data["cl"]), "\n")
print(data["attr 1"], "\n")

0    1
1    1
2    1
3    0
4    0
5    1
6    1
7    1
8    0
9    0
Name: cl, dtype: int64 

[1, 1, 1, 0, 0, 1, 1, 1, 0, 0] 

{0, 1} 

0    1
1    1
2    0
3    1
4    1
5    0
6    1
7    1
8    0
9    0
Name: attr 1, dtype: int64 



Hint: How to split data (Pandas DataFrame) base on column:

In [5]:
data[data['cl']==0]

,attr 1,attr 2,attr 3,attr 4,attr 5,cl
3,1,0,1,0,1,0
4,1,0,0,1,1,0
8,0,1,0,0,1,0
9,0,0,0,1,1,0


Hint: How to take values from column (Pandas Series):

In [6]:
for id, row in data['cl'].items():
    print(id,row)

0 1
1 1
2 1
3 0
4 0
5 1
6 1
7 1
8 0
9 0


1.3 )  Finish the below function for calculating entropy. $H(CL) = - \sum_{y \in CL}p(y)log_2p(y)$ It should return a value of entropy for an input vector CL. Assume that $log_2(0)$ is equal to 0.

In [7]:
def getEntropy(cl) -> float:
    # todo - change; cl is not a list
    entropy = 0.0
    cl_list  = list(cl)
    count0 = cl_list.count(0)
    count1 = cl_list.count(1)
    if len(cl_list):
        if count0:
            entropy -= count0/len(cl_list) * np.log2(count0/len(cl_list))
        if count1:
            entropy -= count1/len(cl_list) * np.log2(count1/len(cl_list))

    return entropy

1.4 ) Calculate the entropy for the CL vector  (the result should be 0.97095...):

In [8]:
print(getEntropy(list(data["cl"])))

0.9709505944546686


1.5) Finish the below function for calculating a conditional entropy: $H(CL|X) = - \sum_{x \in X} \sum_{y \in CL} p(x,y) log_2 \frac{p(x,y)}{p(x)}$. Assume that $log_2(0)$ is equal to 0 and if $p(x) = 0$, $\frac{p(x,y)}{p(x)}$ is equal to 0 as well.

In [9]:
def getConditionalEntropy(cl, attr) -> float:
    entropy = 0.0
    cl_list = list(cl)
    attr_list = list(attr)
    attr0 = list()
    attr1 = list()
    for i in range(len(cl_list)):
        if attr_list[i] == 1:
            attr1.append(cl_list[i])
        else:
            attr0.append(cl_list[i])
    if len(attr0):
        entropy += len(attr0) / len(cl_list) * getEntropy(attr0)
    if len(attr1):
        entropy += len(attr1) / len(cl_list) * getEntropy(attr1)

    return entropy

1.6 ) Calculate conditional entropy for given attributes.

In [10]:
print(getConditionalEntropy(data["cl"], data["attr 1"])) ### the result should be 0.95097...
print(getConditionalEntropy(data["cl"], data["attr 5"])) ### the result should be 0.97095...

0.9509775004326937
0.9709505944546686


1.7 ) **Question: Which entropy is lower and why?**

**Answer** - Conditional entropy of attribute 1 is lower. That's because division of objects based on attribute 5
creates an empty set and set with all objects - i.e., attribute 5 cannot help us differentiate given objects in any way.

1.8) Finish the below function for calculating information gain (use getEntropy() and getConditionalEntropy() functions):

In [11]:
def getInformationGain(cl, attr) -> float:
    return getEntropy(cl) - getConditionalEntropy(cl, attr)

In [12]:
print(getInformationGain(data["cl"], data["attr 1"]))
print(getInformationGain(data["cl"], data["attr 5"]))

0.01997309402197489
0.0


1.9) **Question: Which IG is lower and why?**

**Answer** - IG of attr 5 is lower, because as stated above, it doesn't differentiate objects in any way - i.e. it's
conditional entropy is the same as "just" entropy, thus creating a 0.

# Part 2: ID3 algorithm

Decision tree consists of decision nodes and leaves. Nodes split data while leaves classify objects. Consider the class "Node" provided below. It consists of 4 fields:
- attr - attribute ID (use the names in attributeNames vector)
- left - left branch, i.e., a reference to other node
- right - right branch, i.e., a reference to other node
- value - a decision. If node = None, then the node is not a leaf. If value is not None, then a node is considered a leaf. 

Method __call__ returns the decision if the node is a leaf (i.e., when value is not None). 
Otherwise, it calls either the left or the right branch of an input object, based on the attribute value (0 -> left children; 1 -> right children). In this way, we can traverse the decision tree in order to find the final decision.

In [13]:
class Node:
    def __init__(self, attr, left, right, value):
        self.attr = attr
        self.left = left
        self.right = right
        self.value = value

    def __call__(self, obj):
        if self.value is None:
            if obj[self.attr] == 0:
                return self.left(obj)
            else:
                return self.right(obj)
        else:
            return self.value
        
### EXAMPLE
def example(obj):
    root = Node(0, None, None, None) ###  IN ROOT SPLIT ON 1ST (0) ATTRIBUTE
    lChildren = Node(1, None, None, None) ### IN ROOT's LEFT CHILDREN SPLIT ON 2ND (1) ATTRIBUTE
    rChildren = Node(None, None, None, 2) ### IN ROOT's RIGHT CHILDREN -> DECISION = 2
    root.left = lChildren
    root.right = rChildren
    llChildren = Node(None, None, None, 3) ### IN ROOT's LEFT-LEFT CHILDREN -> DECISION = 3
    lrChildren = Node(None, None, None, 4) ### IN ROOT's LEFT-RIGHT CHILDREN -> DECISION = 4
    lChildren.left = llChildren
    lChildren.right = lrChildren
    print(root(obj))
    
example([0, 0]) ### ROOT : FIRST ATTRIBUTE = 0 SO WE GO TO LEFT CHILDREN.
### IT IS A LEAF WITH THE DECISION = 3
### THEN, IN THE CORRESPONDING CHILDREN, THE SECOND ATTRIBUTE = 0, SO WE GO TO LEFT-LEFT CHILDREN

example([0, 1]) 
example([1, 0])
example([1, 1])

3
4
2
2


2.1) Create an initial root. Set the value (decision) to 1. 

In [14]:
ini_root = Node(None, None, None, 1)

2.2) Use a getErrorRate method in common.py auxiliary file to calculate the error rate. The decision is made based on the majority rule. In case of tie, the method takes 0 as the default class.

In [15]:
cm.getErrorRate(ini_root, data)
## SHOULD BE 0.4

2.3) Use printGraph method (see the common.py file) to draw the decision tree and save it in a png file.

In [20]:
cm.printGraph(ini_root, data)

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

2.4) Calculate information gain for all attributes.

In [76]:
def printInformationGain(data):
    for attribute_name in attributeNames:
        print(attribute_name, getInformationGain(data["cl"], data[attribute_name]))
        
printInformationGain(data)

attr 1 0.01997309402197489
attr 2 0.0464393446710154
attr 3 0.12451124978365313
attr 4 0.0912774462416801
attr 5 0.0


2.5) Choose the best attribute to split the data (HINT, it should be the third attribute :)). Construct two new nodes: one for $x_i$ = 0 decision and the second for $x_i$ = 1; connect them with the root (left and right branch). Remember to update the root. 

In [77]:
ini_root.attr = 2
ini_root.value = None
l = Node(None, None, None, 0)
r = Node(None, None, None, 1)
ini_root.left = l
ini_root.right = r

2.6) Print the graph and calculate the error rate. What happened with the error rate?

In [78]:
print(cm.getErrorRate(ini_root, data))
# Answer: Error Rate became lower, because we "gained information" from division based on attribute 3

cm.printGraph(ini_root, data)

0.30000000000000004


KeyError: 2

2.7) Split the 'data' (table) based on the selected attribute, i.e., create two new tables.

In [79]:
left_data = data[data["attr 3"] == 0]
right_data = data[data["attr 3"] == 1]
print(left_data)
print(right_data)

   attr 1  attr 2  attr 3  attr 4  attr 5  cl
1       1       1       0       0       1   1
4       1       0       0       1       1   0
7       1       0       0       1       1   1
8       0       1       0       0       1   0
9       0       0       0       1       1   0
   attr 1  attr 2  attr 3  attr 4  attr 5  cl
0       1       0       1       1       1   1
2       0       1       1       1       1   1
3       1       0       1       0       1   0
5       0       0       1       1       1   1
6       1       1       1       1       1   1


2.8) Let us start with the left node. Firstly, calculate information gain for this node.

In [80]:
printInformationGain(left_data)

attr 1 0.4199730940219749
attr 2 0.01997309402197489
attr 3 0.0
attr 4 0.01997309402197489
attr 5 0.0


2.9) Choose the best attribute to split the data and then update the decision tree.

In [81]:
# attr 1 is the best choice
ll = Node(None, None, None, 0)
lr = Node(None, None, None, 1)
l = Node(0, ll, lr, None)
ini_root.left = l

2.10) Print the graph and calculate the error rate (HINT: should be 0.2 :)). What happened with the error rate?

In [82]:
print(cm.getErrorRate(ini_root, data))
# Answer: Error Rate became even lower, because we "gained even more information" from division based on attribute 1 in
# the left node

cm.printGraph(ini_root, data)

0.19999999999999996


KeyError: 2

2.11) Split data (remember that we split left_data, not data).

In [83]:
leftLeft_data = left_data[left_data["attr 1"] == 0]
leftRight_data = left_data[left_data["attr 1"] == 1]
print(leftLeft_data)
print(leftRight_data)

   attr 1  attr 2  attr 3  attr 4  attr 5  cl
8       0       1       0       0       1   0
9       0       0       0       1       1   0
   attr 1  attr 2  attr 3  attr 4  attr 5  cl
1       1       1       0       0       1   1
4       1       0       0       1       1   0
7       1       0       0       1       1   1


2.12) Repeat the whole process for the right node.

In [84]:
printInformationGain(right_data)

attr 1 0.17095059445466865
attr 2 0.17095059445466865
attr 3 0.0
attr 4 0.7219280948873623
attr 5 0.0


In [85]:
# attr 4 is the best choice
rl = Node(None, None, None, 0)
rr = Node(None, None, None, 1)
r = Node(3, rl, rr, None)
ini_root.right = r

In [86]:
print(cm.getErrorRate(ini_root, data))
# Answer: Error Rate became even lower, because we "gained even more information" from division based on attribute 4 in
# the right node

cm.printGraph(ini_root, data)

0.09999999999999998


KeyError: 2

In [87]:
# TODO split the data (right_data)
rightLeft_data = right_data[right_data["attr 4"] == 0]
rightRight_data = right_data[right_data["attr 4"] == 1]
print(rightLeft_data)
print(leftRight_data)

   attr 1  attr 2  attr 3  attr 4  attr 5  cl
3       1       0       1       0       1   0
   attr 1  attr 2  attr 3  attr 4  attr 5  cl
1       1       1       0       0       1   1
4       1       0       0       1       1   0
7       1       0       0       1       1   1


2.13) Let's consider left-left node. Calculate information gain for it.

In [99]:
printInformationGain(leftLeft_data)

attr 1 0.0
attr 2 0.0
attr 3 0.0
attr 4 0.0
attr 5 0.0


2.14) Will adding a new node to the tree improve its effectiveness? Why? Why not?

**Answer** - no, because there's no IG from using any of the attributes

2.15) Calculate information gain for the left-right node.

In [100]:
printInformationGain(leftRight_data)

attr 1 0.0
attr 2 0.2516291673878229
attr 3 0.0
attr 4 0.2516291673878229
attr 5 0.0


In [101]:
### Select the attribute and update the tree

# both attr 2 and attr 4 are equally good
lrl = Node(None, None, None, 0)
lrr = Node(None, None, None, 1)
lr = Node(1, lrl, lrr, None)
l = Node(l.attr, ll, lr, None)
ini_root.left = l

In [102]:
### Print the decision tree and compute the error rate
print(cm.getErrorRate(ini_root, data))
cm.printGraph(ini_root, data)

0.09999999999999998


KeyError: 2

2.16) What happened with the error rate? Is it necessary to keep these two newly added leaves?

**Answer** - error rate didn't change, so these two newly added leaves are *useless*.

2.17) Finish creating the right side of the tree

In [103]:
printInformationGain(rightLeft_data)
print("since IG is 0 for all attributes, there's no need for division\n")

printInformationGain(rightRight_data)
print("since IG is 0 for all attributes, there's no need for division\n")

attr 1 0.0
attr 2 0.0
attr 3 0.0
attr 4 0.0
attr 5 0.0
since IG is 0 for all attributes, there's no need for division

attr 1 0.0
attr 2 0.0
attr 3 0.0
attr 4 0.0
attr 5 0.0
since IG is 0 for all attributes, there's no need for division



# Part 3: automated construction of decision trees

3.1 Complete the following function for automated construct of decision trees, so that it returns a decision tree for the given data and attribute list. Note that this is a recursive method, i.e., calls itself.

In [ ]:
max_depth = 0

def createTree(data, attributeNames, depth=0):
    data = data.reset_index().drop("index", axis=1)
    ### TODO

3.2) Build a decision tree for a training dataset in the common.py auxiliary file, for different values of max_depth.  Calculate & compare the error rates for training and validation datasets.

In [ ]:
max_depth = 10

In [ ]:
# Training dataset
train_attributeNames, train_data = cm.getTrainingDataSet()
### TODO

In [ ]:
# Validation dataset
valid_attributesName, valid_data = cm.getValidationDataSet()
### TODO

3.3) Consider only the training data set and answer the following questions:
* What is the minimum depth of the tree (consider only the training data set)?
* The tree building process should stop when there is no improvement in error rate (why?). Check for which value of "max_dept" there is no improvement in error rate. 

In [ ]:
for i in range(10):
    max_depth = i
    ### TODO